In [ ]:
#=========================================
# ECC Code Structure
#=========================================

# === Imports ===
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from pathlib import Path
import os

# === Settings ===
input_folder = "Input Folder Path" #<------ Update File path
output_folder = "Outout Folder Path" #<------ Update File path
Path(output_folder).mkdir(exist_ok=True)

gaussian_blur_kernel = (5, 5)

# === ECC Alignment Function ===
def align_image_ecc(ref_img, target_img):
    ref = ref_img.astype(np.float32) / 255.0
    tgt = target_img.astype(np.float32) / 255.0

    if ref.shape != tgt.shape:
        print("ECC: Image sizes do not match.")
        return None

    warp_matrix = np.eye(2, 3, dtype=np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 500, 1e-6)

    try:
        cc, warp_matrix = cv2.findTransformECC(
            ref, tgt, warp_matrix,
            motionType=cv2.MOTION_AFFINE,
            criteria=criteria,
            inputMask=None
        )

        aligned = cv2.warpAffine(
            target_img, warp_matrix,
            (ref_img.shape[1], ref_img.shape[0]),
            flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP
        )
        return aligned

    except cv2.error as e:
        print(f"ECC alignment failed: {e}")
        return None

# === Heatmap Plotting Function ===
def plot_heatmap(diff_img, img_name, output_path):
    try:
        # Assume diff_img is already float32 and signed
        diff_centered = diff_img
        plt.figure(figsize=(10, 6))
        im = plt.imshow(diff_centered, cmap='seismic', vmin=-256, vmax=256)
        plt.colorbar(im, label="Pixel Intensity Difference")
        plt.title(f"Difference Heatmap\n{img_name}")
        plt.axis("off")
        plt.tight_layout()
        plt.savefig(output_path, dpi=300)
        plt.close()

    except Exception as e:
        print(f"Error plotting heatmap for {img_name}: {e}")

# === Main Loop ===
image_paths = sorted(glob(os.path.join(input_folder, "*.bmp"))) #<------ CHANGE THIS FOR DIFFERENT IMAGE TYPES - WILL NOT WORK IF NOT MATCHING
assert len(image_paths) >= 2, "You need at least two images to compare!"

ref_img = cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)
ref_img = cv2.GaussianBlur(ref_img, gaussian_blur_kernel, 0)

for i in range(1, len(image_paths)):
    tgt_path = image_paths[i]
    tgt_img = cv2.imread(tgt_path, cv2.IMREAD_GRAYSCALE)
    tgt_img = cv2.GaussianBlur(tgt_img, gaussian_blur_kernel, 0)

    img_name = f"{Path(tgt_path).stem} - {Path(image_paths[0]).stem}"

    aligned = align_image_ecc(ref_img, tgt_img)

    if aligned is None:
        print(f"Skipping {img_name} due to alignment failure.")
        continue

    # === Image Subtraction  ===
    diff_img = aligned.astype(np.float32) - ref_img.astype(np.float32)

    # Plot heatmap with red/white/blue centered at 0
    heatmap_path = f"{output_folder}/heatmap_{img_name}.png"
    plot_heatmap(diff_img, img_name, heatmap_path)

    print(f"Processed: {img_name}")

print("ECC batch alignment complete!")